# Cisco Blueprint

* 3.2 __Layer 3 (ENCOR)__ https://learningnetwork.cisco.com/s/encor-exam-topics
    * 3.2.a Compare routing concepts of EIGRP and OSPF (advanced distance vector vs. linked state, load balancing, path selection, path operations, metrics)

* 1.9 __Troubleshoot EIGRP__ (classic and named mode) (ENARSI) https://learningnetwork.cisco.com/s/enarsi-exam-topics
    * 1.9.a Address families (IPv4, IPv6)
    * 1.9.b Neighbor relationship and authentication
    * 1.9.c Loop-free path selections (RD, FD, FC, successor, feasible successor, stuck in active)
    * 1.9.d Stubs
    * 1.9.e Load balancing (equal and unequal cost)
    * 1.9.f Metrics

* 1.3 __EIGRP (Enterprise Lab)__ https://learningnetwork.cisco.com/s/ccie-enterpr-infrastructure-exam-topics
    * 1.3.a Adjacencies
    * 1.3.b Best path selection
        * 1.3.b i RD, FD, FC, successor, feasible successor
        * 1.3.b ii Classic Metrics and Wide Metrics
    * 1.3.c Operations
        * 1.3.c i General operations
        * 1.3.c ii Topology table
        * 1.3.c iii Packet types
        * 1.3.c iv Stuck In Active
        * 1.3.c v Graceful shutdown
    * 1.3.d EIGRP load-balancing
        * 1.3.d i Equal-cost
        * 1.3.d ii Unequal-cost
        * 1.3.d iii Add-path
    * 1.3.e EIGRP Named Mode
        * 1.3.f Optimization, convergence and scalability
        * 1.3.f i Fast convergence requirements
        * 1.3.f ii Query propagation boundaries
        * 1.3.f iii IP FRR (single hop)
        * 1.3.f iv Leak-map with summary routes
        * 1.3.f v EIGRP stub with leak map

## Other ref
* __Cisco Ref__ : https://www.cisco.com/c/en/us/support/docs/ip/enhanced-interior-gateway-routing-protocol-eigrp/16406-eigrp-toc.html
* __RFC__ : https://tools.ietf.org/html/rfc7868
* __DUAL__ : https://escholarship.org/uc/item/0j73185m

# Theory 
## From RIP to IGRP 
EIGRP was developped during 1980s to overcome the limitation of RIP, the main being 
the __hop count metric with max 15 hops network diameter__ , However both being of type DVRP share a handful charactesistics 

|Similarities | Difference |
|--- | ---|
|classful | weighted composit metric |
|DVRP | 255 msx diameter |
| periodic broadcast | uniqual cost LB |
| split horizon as loop-prevension | 3X update period than RIP  |
| sanity check : neighbours must share same subnet | More efficient update packets |
| triggered updates + hold-down timers | IP protocol 9 rather UDP |

### Process domains 
Similar to OSPF, IGRP could run __multiple simultainus instances__, identified by process ID. Exchange of routes between process domains is done by __Redistribution__. Only boundaty routers could perform route summarization. IGRP had 3 routes types,
1. Interior : within a process domain 
2. System : Summary routes 
3. Exterior : Between process domains or other autonomous systems or protocols 

### Timers
|Type | duration |
|--- | ---| 
|Update  $t_u$| 90 sec - 20% jitter (for avoiding sync) = [72 - 90]sec |
|Invalid $t_i$| 3 X $t_u$ = 270 sec | 
|Flush $t_f$| 7 X $t_u$ = 630 sec |  

If  router dosn't recieve update from an existing route within $t_i$ it marks it unreachable untill $t_f$ expires, after that, it removes it from the routing table. A longer timer is good as it yields __better link utilization__ than RIP and bad for its __slow convergence__. 

### Metric 
| parameters | usage |
| ---- | --- |
| Bandwith + delay | link capacity | 
| relybility + load | link dynamics |
| MTU | Tie-breaker | 

$Metric_{igrp} = \bigg[ k_1 min_i \bigg( \frac{10^7}{BW_i} \bigg) + 
                 k_2 \bigg\{ \frac{ min_i \bigg( \frac{10^7}{BW_i} \bigg) } { 256-load} \bigg\} + 
                 k_3 \sum_i \bigg( \frac{DLY_i}{10} \bigg)\bigg] +
                 \bigg( \frac{k_5}{RLY + k_4} \bigg)\\
                 i \in Path_{s,d}$
                 
                 
by default $k_1=k_3=1$ and $k_2=k_4=k_5=0$ thus, $Metric_{igrp}=\frac{10^7}{min(BW)}+\frac{\sum dealy}{10}$

## From IGRP to EIGRP 
The original proposal EIGRP was to __make IGRP classless__ keeping it a typical DVRP but, designers chose to make it a hybrid by using some features from LSRP. 

|Distance vector routing | Link state routing |
| ---- | ---- |
| shares every info with direct neighbours | share info learned from direct neighbours to everybody |  
| core algo : Bellman-Ford | core algo : Dijkstra's SPF |
| Prune to Routing loops and count-to-infinity problem | Less prune to routing loops | 
| Loop prevension methods : split horizon + route poisoning | no loop prevension needed | 
| Due to additional loop prevension overhead, slow convergence in large networks | faster convergence but huge database maintainance | 
| calculate cost then send updates | accumulates all updates then calculate costs | 

EIGRP was developped as a DVRP that wors as a LSRP. In both the cases, route calculation is a distributed algorithm and __each router calculates indipendently__. EIGRP is classified as _Advanced Distance Vector Routing protocol (ADVRP)_ as it makes uses the __Diffusion Computation__ technique, that makes a __distributed computation collaborative__, which reduces times complexity of DVRP and space complexity of LSRP. 

### EIGRP Updates
* Vector of distences : list of metric calculated for all known destination networks 
* Non-Periodic : Event driven (only sent if topology/metric changes)
* Partial : Except first time, only affected routes are sent 
* Bounded : Updates are only sent to affected routers
* Regulated : EIGRP updtes uses at most 50% of the link bandwidth by default, to alter use `(int)# ip bandwidth-percent eigrp [num]` command. 
* Classless : Sends subnet masks in updates and support aggregation 
* Secure : uses MD5 authentication to make ensure neighbours before sending updates 

### EIGRP Metric
$Metric_{eigrp} = 256 \times Metric_{igrp} = 256\times\bigg[ k_1 min_i \bigg( \frac{10^7}{BW_i} \bigg) + 
                 k_2 \bigg\{ \frac{ min_i \bigg( \frac{10^7}{BW_i} \bigg) } { 256-load} \bigg\} + 
                 k_3 \sum_i \bigg( \frac{DLY_i}{10} \bigg)\bigg] +
                 \bigg( \frac{k_5}{RLY + k_4} \bigg)\\
                 i \in Path_{s,d}$
                 
                 
by default $k_1=k_3=1$ and $k_2=k_4=k_5=0$ thus, $Metric_{igrp}=256\times\bigg[\frac{10^7}{min(BW)}+\frac{\sum dealy}{10}\bigg]$

use the command `(router)# metric weight [k1] [k2] [k3] [k4] [k5] [k6]` to adjust the values. 

### EIGRP Protocol architecture 
EIGRP uses a stack of modules in its architecture that bridges it in between L4 and L3 in the OSI stack. 

|Modules | Operation |
|--- | --- |
|Transport layer | L4 encapsulation into segments | 
|Protocol Dependent modules | Interfaces with IP, IPX and Appletalk modeules for interoperability | 
|DUAL | Core algorithm to compute routes | 
|Neighbourship Mechanism | Performs neighbour discovary and make adjacecny |
|RTP | reliable transport protocol, esnsures reliability control traffic | 
|Network layer | L3 encapsulation into packets | 

A buttom-up understanding of the modules are given below, 
1. __RTP__ : Reliable Trasport Protocol is Cisco's propreatary mechanism for assuring delivary of EIGRP control packets.
    * it uses cisco's own algorithm called __Reliable Multicast__ for neighbour discovary at 224.0.0.10 (reserved) address
    * Ensures no out-of-order packets using 2 sequence numbers
    * If a Reliable-Multicast packet doesnt get any acklowledgement untill __MC Flow timer__ expires it switches to Reliable-Unicast and keep sending 16 unicast packets in an interval called __Retransmission Timeout (RTO)__, after which the subjected neighbour is discarded.  
    * MC-flow timers and RTO is calculted from __Smooth RTT (SRTT)__ which calculated as $SRTT = f\bigg(\frac{\text{RM Tx timestamp} + \text{Ack Rx timestamp} }{2}\bigg)$, the actual $f$ is propriatary to cisco
    * 6 differenct control packets 
    
|packets | Reliable | Unreliable | Unicast | Multicast | 
| ---    | ---      | ---        | ---     | ---       |
| Hello  |          | X          |         | X         |
| Ack    |          | X          |X        |           |
| Update | X        |            |X        | X         |
| Query  | X        |            |X        | X         |
| Reply  | X        |            |X        |           |

2. __Neighbourship Mechanism__ : 
    * Hello packets is sent on 5 Sec interval in Broadcast/unicast networks and 60 sec over NBMA networks. Alter this using `(int)# ip hello-interval eigrp [num]`
    * Hold-timer is included into hello packets, it tells the recipient to consider the sender dead after the hold-time. Alter this using `(int)# ip hold-time eigrp [num]`
    * EIGRP maintains a neighbour table to keep the neighbour information 
    
|H (handle)| Address | Interface | Hold | Uptime | SRTT | RTO | Q count | Seq |
|---|---|---|---|---|---|---|---|---|
order of discovary | dest. prefix | exit iface | hold-down | uptime | SRTT | RTO | number of packet queued | from RTP |

![](pics/eigrp1.png)

3. __Diffusion Update ALgorithm (DUAL)__
    * Convergence algorithm , replaces bellman ford. 
    * The core design philosophy of DUAL was, _"even temporal routing loops are deteminental to the performace of the network"_.
    * Firt proposed by Dijkstra to develop distributed SPF but later implemented by Garcia. 
    * DUAL relies on the underlying protocols for the following (RTP ensures them)
        * Neighbour connectivity and status must be repoted in a finite time.
        * Reliable transport of control packets.
        * All event are processed sequentially with a deterministic order of receiving them.
        
### DUAL Algorithm 
1. __Initializarion__ : start sending hello to discover neighbour and to identify itself to the neighbours. 
2. __Adjacency__ : An adjacency is a logical link over which routing information are exchanged. Adjacency conditions are (identical k values, ASN). Once eshtablished adjacencies, router starts receiving updates form neighbours.Update contains __routes__ known by each neighbours and their associated __metric__ , also called __Repoted distance (RD)__. 
3. __Route Calculation__ : For each routes (destinatio prefix) router starts calculating optimal routes. the neighbour(s) with __minimum metric__ becomes the __successor__ and the subhected metric is called __Feasible Distance (FD)__. Backup paths are reserved by setting neighbours as __Feasible Successor (FS)__ if they satisfy the __Feasibility condition (FC)__ i.e. $RD(Prefix)<FD(prefix)$
4. __Load Balancing__ : All successor would perform __equal-cost__ load-balancing, only feasible successors can take part of __unequal-cost__ load balancing defined by __varience__ value.  

Dual populates two tables.
1. __Topology table__ : similar to OSPF LSDB, topology tables.
2. __Routing table__ : Only feasible route(s) make thei way to Routing table. 
       
| desination prefix | FD | All FS | RD of each FS | Distacne through each FS | exit iface |
|---|---|---|---|---|---|
| Network ID | least distance | backup neighbours | FD from Neighbours | FD through neighbours | iface to FS |

       

### DUAL FSM
When an EIGRP rouer is not performing DUAL its interfaces are kept in __Passive__ state. A router reassesses its list of FS for a route, 
* __Listen to event__ : if any of the following events take place 
    1. change in cost of directly connected link
    2. Flapping of directly connected link
    3. Reception of Update / Query / Reply 
* The router first performs local computation of distance for FS
* __Case 1: FS found__
    1. If any FS offers lowest distance than FD, FS becomes successor and updates FD
    2. Send update to neighbours 
* __Case 2: FS not found__
    1. Enter into diffusion update, this mark the route into __Active__ state. Until it goes back to passive state, the router can't Add or update the corresponding RD/FD nor can start another DUAL.    
    2. Start sending __Querry__ message to its neighbours. Since there are multple reasons that turn an route into active state, EIGRP defines multiple (4) active states, identified in the __Query Origin Flag__.   
    3. If neighbours have a FS to the querried route it __Replies__ with new RD, otherwise mark itself into Active state. 
* __Termination__ : DUAL terminates when replies to __All Queries__ are received. During DUAL, router set FD=inifity, thus any neighbour replying with a finite metric would meet FC.  

__Stuck in Active (SIA)__ : For large networks with low Throughput and/or high latency, a router may not receive replies to all queries it sent. A router waits untill __Active Timer = 3 mins (default)__ before it decaires the route __SIA__, the route is not removed but __set to infinite metric__. use `(router)# timer active-time [min | disabled]` to control the timer behaviour. In a well-designed network SIA should not happen, two more messages namely, __SIA-Query & SIA-Reply__ reduce the chance of SIA occurance and easy the troupleshooting.

### EIGRP Packet formats

EIGRP header is defined by __protocol number 88__, the packet size is limited by MTU = 1500B default. The header contains a fixed sized EIGRP header (20B) there's a variable length array of __Type/len/value (TLV)__ triplet (4B). TLV carries EIGRP management info agnostic to any underlaying protocol  

|Header Type | Field name | size | Purpose |
|---         | ---        | ---  | ---     |
|Fixed       | Version    | 1    | 2       |
|            | Opcode     | 1    | 1=upd, 3=qry, 4=rply, 5=hello, 10=SIA_q, 11=SIA_r          |
|            | Checksum   | 2    | IP Checksum        |
    |            | Flag       | 4    | __Init__ : full update, __Cond. rcv__: prop. multicast, __Rst__, __End of table__        |
|            | Sequence   | 2    | RTP Sequence number        |
|            | Ack        | 2    | RTP Ack number        |
|            | ASN        | 2    | AS number        |

* Follwing is the TLV (__eigrp parameter__ header) 

|Header Type | Field name | size | Purpose |
|---         | ---        | ---  | ---     |
|TLV (param) | Type       | 2    | 0x0001 (EIGRP param)         |
|            | Len        | 2    | length of packet |
|            | K1 - K5    | 2X5  | k values |
|            | Reserved   | 16    | reserved |
|            | Hold time  | 16    | Hold time |

* Following is the TLV (__Internal route__ header) 

|Header type | Field name | size | Purpose |
|---         | ---        | ---  | ---     |
|TLV (D)     | type       | 2    | 0x0102 (Internal Routes) |
|            | Len        | 2    | length of the packet | 
|            | Next hop   | 4    | next hop IP address |
|            | Delay      | 4    | 0xFFFFFFFF = infinity, unreachable route |
|            | Bandwidth  | 4    | Bandwidth |
|            | MTU        | 3    | MTU|
|            | Hopcount   | 1    | 255 maximum hop count |
|            | Reliability| 1    | 0x01=unreliable ... 0xFF=100% reloable |
|            | Load       | 1    | 0x01=no load ... 0xFF = fully loaded | 
|            | Reserved   | 2    | |
|            | prefix len | var    | number of network bits |
|            | Dest       | var    | destination prefix | 

* Following is the TLV (__External route__ header) 

|Header type | Field name | size | Purpose |
|---         | ---        | ---  | ---     |
|TLV (D EX)  | type       | 2    | 0x0103 (Internal Routes) |
|            | Len        | 2    | length of the packet | 
|            | Next hop   | 4    | next hop IP address |
|            | O Rtr      | 4    | IP/RID of the redistributing router | 
|            | O ASN      | 4    | Original ASN |
|            | Arb. tag   | 4    | used by route tagging |
|            | Ext metric | 4    | external protocol metric | 
|            | Reserved   | 2    | |
|           | Ext proto ID| 1    | $0\times 0x$ : 2=EIGRP, 3=Static, 4=RIP, 5=Hello, 6=OSPF, 7=IS-IS, 9=BGP, B=Con  |
|            | Flags      | 1    | 0x01 : external , 0x02 : candidate default |  
|            | Delay      | 4    | 0xFFFFFFFF = infinity, unreachable route |
|            | Bandwidth  | 4    | Bandwidth |
|            | MTU        | 3    | MTU|
|            | Hopcount   | 1    | 255 maximum hop count |
|            | Reliability| 1    | 0x01=unreliable ... 0xFF=100% reloable |
|            | Load       | 1    | 0x01=no load ... 0xFF = fully loaded | 
|            | Reserved   | 2    | |
|            | prefix len | var    | number of network bits |
|            | Dest       | var    | destination prefix | 

# Configuration 
![](pics/eigrp2.png)
## Base Config 
```
conf t
    router eigrp [asn]        ! activate eigrp process with given ASN
    no auto-summary           ! stop automatic address summarization
    passive-interface default ! put all interface into passive state  
    
    net [int-ip] 0.0.0.0      ! advertise network  
    
    no passive-interface [iface-id] ! select active interfaces 
exit
```


## Uneqal cost Load Balancing 
Load balance between R1 and R3. 
```
!r3
conf t
    router eigrp 200
    varience [x]   ! calculate x as ceil( Dist(FS)/FS )
exit
```

## Setting Maximum Path

## Multiple EIGRP Process

## Manual Summary 

## Stub routing 

## Authentication 